In [ ]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM


In [ ]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 8

In [ ]:
# Embedding dimensions.
row_hidden = 128
col_hidden = 128

In [ ]:

# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Reshapes data to 4D for Hierarchical RNN.

# HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence. Usually, the first recurrent layer of an HRNN encodes a sentence
# (e.g. of word vectors) into a sentence vector. The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a
# document vector. This document vector is considered to preserve both the word-level and sentence-level structure of the context.

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))




In [ ]:
# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistributed. This wrapper applies a layer to every temporal slice of an input. The input should be at least 3D, and the dimension of index one will be
# considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)




In [ ]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



Epoch 1/8
1875/1875 [==============================] - 23s 11ms/step - loss: 0.4903 - accuracy: 0.8346 - val_loss: 0.1283 - val_accuracy: 0.9617
Epoch 2/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1176 - accuracy: 0.9643 - val_loss: 0.0665 - val_accuracy: 0.9789
Epoch 3/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0774 - accuracy: 0.9768 - val_loss: 0.0515 - val_accuracy: 0.9847
Epoch 4/8
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0608 - accuracy: 0.9820 - val_loss: 0.0600 - val_accuracy: 0.9814
Epoch 5/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0521 - accuracy: 0.9840 - val_loss: 0.0446 - val_accuracy: 0.9868
Epoch 6/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0441 - accuracy: 0.9868 - val_loss: 0.0435 - val_accuracy: 0.9872
Epoch 7/8
1875/1875 [==============================] - 20s 10ms/step - loss: 0.0379 - accuracy: 0.9883 - val_loss: 0.0448 - val_ac

In [ ]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0]*100,"%")
print('Test accuracy:', scores[1]*100,"%")

Test loss: 4.545249789953232 %
Test accuracy: 98.83000254631042 %
